In [1]:
%%capture
!pip install transformers==4.39.3
!pip install -U datasets==2.16.0
!pip install -U unlimited-classifier==0.0.2

In [2]:
from datasets import load_dataset

ds=load_dataset("dair-ai/emotion")
ds

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
classes=ds['test'].features["label"].names
idx2class={idx:class_ for idx, class_ in enumerate(classes)}

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

model_name = 'knowledgator/flan-t5-base-for-classification'
device='cuda'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

2024-04-05 00:03:37.354261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 00:03:37.354412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 00:03:37.488334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
from unlimited_classifier.main import TextClassifier
from tqdm import tqdm

classifier = TextClassifier(
   labels=classes,
   model=model,
   tokenizer=tokenizer,
   device=device,
   num_beams=1
)


prompt = """ Classify the following text and return the emotion name that belongs to it.
Text:
"""

batch_size = 8

results = []
for id in tqdm(range(0, len(ds['test']), batch_size)):
    examples = ds['test'][id:id+batch_size]
    texts = [prompt + text for text in examples['text']]
    output = classifier.invoke_batch(texts)
    predicts = [res[0] for res in output]
    results+=predicts

Compiling /opt/conda/lib/python3.10/site-packages/unlimited_classifier/labels_trie.pyx because it changed.
[1/1] Cythonizing /opt/conda/lib/python3.10/site-packages/unlimited_classifier/labels_trie.pyx


100%|██████████| 250/250 [00:20<00:00, 12.30it/s]


In [7]:
from sklearn.metrics import classification_report

class2idx = {class_:idx for idx, class_ in idx2class.items()}
conv_results = [class2idx[res[0]] for res in results]
labels = ds['test']['label']
print(classification_report(labels, conv_results, target_names=classes, digits=4))

              precision    recall  f1-score   support

     sadness     0.6667    0.7470    0.7045       581
         joy     0.7953    0.6763    0.7309       695
        love     0.4500    0.2830    0.3475       159
       anger     0.5538    0.6545    0.6000       275
        fear     0.5466    0.7589    0.6355       224
    surprise     0.7273    0.2424    0.3636        66

    accuracy                         0.6575      2000
   macro avg     0.6233    0.5604    0.5637      2000
weighted avg     0.6672    0.6575    0.6520      2000

